In [1]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

In [2]:
!pip install -qqq datasets Pillow==9.4.0 transformers qwen_vl_utils accelerate

In [3]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xm3600_captioning": "data/xm3600_captioning-00000-of-00001.parquet"}
)

In [4]:
ds['xm3600_captioning'][0]

{'sample_id': 1,
 'image_id': '000411001ff7dd4f',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 'prompt_en': 'Caption the image, short answer.',
 'captions_en': ['a rooster and hens surrounded by green leaves .',
  'a rooster with two hens on a rocky slope with some bushes .'],
 'prompt_ar': 'علّق على الصورة، بإجابة قصيرة.',
 'captions_ar': ['ديك و فرخة علي الأرض', 'ديك وفرخة بحديقة'],
 'prompt_bn': 'ছবির ক্যাপশন দিন, সংক্ষিপ্ত উত্তর।',
 'captions_bn': ['জঙ্গলের মদহে দুইটি বাদামী ও কালো রঙের মুরগি আছে'],
 'prompt_cs': 'Popište obrázek krátkou odpovědí.',
 'captions_cs': ['kohout a slepice v trávě',
  'hnědá slepice a kohout jdoucí po trávě v lese'],
 'prompt_da': 'Giv billedet en billedtekst, kort svar.',
 'captions_da': ['en brun høne og en flerfarvet hane i skovbunden',
  'hane og høne i have'],
 'prompt_de': 'Bildunterschrift für das Bild, kurze Antwort.',
 'captions_de': ['eine henne und ein hanh im steinigem garten im gras .',
  'leicht verschwommene 

In [5]:
# Qwen2.5-VL model setup
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

from tqdm import tqdm
from PIL import Image
import os

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_caption", exist_ok=True)

dataset = ds['xm3600_captioning']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        #### This part will be replaced when attack implementation is ready #####

        image_path = f"temp_images_caption/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        #########################################################################

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            prompt_col = f"prompt_{lang}"
            caption_col = f"captions_{lang}"

            # Safety check if caption exists
            if prompt_col not in sample or caption_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            prompt = sample[prompt_col]
            gt_caption = sample[caption_col]

            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image_path},
                        {"type": "text", "text": prompt}
                    ]
                }
            ]

            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            ).to("cuda")

            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False
            )
            pred_caption = output_text[0]

            # Display result
            print(f"\n[{lang.upper()}]")
            print(f"Prompt: {prompt}")
            print(f"GT: {gt_caption}")
            print(f"Pred: {textwrap.fill(pred_caption, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break

model.safetensors.index.json:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Iterating samples:   0%|          | 0/3600 [00:00<?, ?it/s]


========== Sample ID: 1 ==========

[EN]
Prompt: Caption the image, short answer.
GT: ['a rooster and hens surrounded by green leaves .', 'a rooster with two hens on a rocky slope with some bushes .']
Pred: "Three chickens in a garden with greenery and fallen leaves."

[BN]
Prompt: ছবির ক্যাপশন দিন, সংক্ষিপ্ত উত্তর।
GT: ['জঙ্গলের মদহে দুইটি বাদামী ও কালো রঙের মুরগি আছে']
Pred: "Three chickens in a garden."

[DE]
Prompt: Bildunterschrift für das Bild, kurze Antwort.
GT: ['eine henne und ein hanh im steinigem garten im gras .', 'leicht verschwommene aufnahme von huhn und hahn im freien tagsüber .']
Pred: "Ein Hahn und eine Henne in einem natürlichen Umfeld."

[KO]
Prompt: 이미지에 캡션을 달아주세요. 짧은 답변으로.
GT: ['낙엽 과 잡초 가 많은 산길 의 돌 위에 서 있는 닭 두 마리', '시골 마을 에 풀어 놓고 키우는 닭 암수']
Pred: Three chickens are walking on the ground in a garden with green plants and
fallen leaves.

[RU]
Prompt: Подпишите изображение, короткий ответ.
GT: ['Яркий петух и его коричневые куры в парке', 'Курица и петух , бегущие п

Iterating samples:   0%|          | 0/3600 [00:05<?, ?it/s]


[ZH]
Prompt: 为图片添加标题，简短回答。
GT: ['在 山里 中 站着 两只鸡 ， 一只 黄色 另一 只 黑 黄色 ， 它们 俩 站着 看 向 同 一个 方向', '在 野外 绿植 地上 的 公鸡 和 母鸡 近景']
Pred: 鸡在草地上走
